### Beauty Ahmad

In [4]:
beauty_df = pd.read_csv("beauty.csv")


In [5]:
beauty_df.head()

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,Size:
0,0,5,True,2016-09-01,A3CIUOJXQ5VDQ2,B0000530HU,Shelly F,As advertised. Reasonably priced,7.0 oz
1,1,5,True,2013-11-14,A3H7T87S984REU,B0000530HU,houserules18,Like the oder and the feel when I put it on my...,7.0 oz
2,2,1,True,2013-08-18,A3J034YH7UG4KT,B0000530HU,Adam,I bought this to smell nice after I shave. Wh...,7.0 oz
3,3,5,False,2011-05-03,A2UEO5XR3598GI,B0000530HU,Rich K,HEY!! I am an Aqua Velva Man and absolutely lo...,7.0 oz
4,4,5,True,2011-05-06,A3SFRT223XXWF7,B00006L9LC,C. C. Christian,If you ever want to feel pampered by a shampoo...,200ml/6.7oz


In [6]:
del beauty_df["Unnamed: 0"]

In [7]:
beauty_df['reviewTime'] = pd.to_datetime(beauty_df['reviewTime'])
#beauty_df.head()

In [8]:
beauty_df.rename(columns={"overall":"productRating","asin":"productID", "Size:":"size"}, inplace=True)
beauty_df.head()

,productRating,verified,reviewTime,reviewerID,productID,reviewerName,reviewText,size
0,5,True,2016-09-01,A3CIUOJXQ5VDQ2,B0000530HU,Shelly F,As advertised. Reasonably priced,7.0 oz
1,5,True,2013-11-14,A3H7T87S984REU,B0000530HU,houserules18,Like the oder and the feel when I put it on my...,7.0 oz
2,1,True,2013-08-18,A3J034YH7UG4KT,B0000530HU,Adam,I bought this to smell nice after I shave. Wh...,7.0 oz
3,5,False,2011-05-03,A2UEO5XR3598GI,B0000530HU,Rich K,HEY!! I am an Aqua Velva Man and absolutely lo...,7.0 oz
4,5,True,2011-05-06,A3SFRT223XXWF7,B00006L9LC,C. C. Christian,If you ever want to feel pampered by a shampoo...,200ml/6.7oz


In [9]:
beauty_df.groupby('productID').count().sort_values('productRating', ascending = False)

,productRating,verified,reviewTime,reviewerID,reviewerName,reviewText,size
productID,,,,,,,
B0012Y0ZG2,1794,1794,1794,1794,1794,1792,1794
B000URXP6E,858,858,858,858,858,857,857
B001OHV1H4,489,489,489,489,489,488,489
B00006L9LC,489,489,489,489,489,488,489
B0009RF9DW,392,392,392,392,392,392,392
...,...,...,...,...,...,...,...
B00MGK9Z8U,1,1,1,1,1,1,0
B0091OCA86,1,1,1,1,1,1,0
B00B7V273E,1,1,1,1,1,1,1


#### The Data at a glance

In [10]:
beauty_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5269 entries, 0 to 5268
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   productRating  5269 non-null   int64         
 1   verified       5269 non-null   bool          
 2   reviewTime     5269 non-null   datetime64[ns]
 3   reviewerID     5269 non-null   object        
 4   productID      5269 non-null   object        
 5   reviewerName   5269 non-null   object        
 6   reviewText     5264 non-null   object        
 7   size           4358 non-null   object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(5)
memory usage: 293.4+ KB


In [11]:
beauty_df.isnull().sum()

productRating      0
verified           0
reviewTime         0
reviewerID         0
productID          0
reviewerName       0
reviewText         5
size             911
dtype: int64

 I renamed the columns to clearer terms, I also figured I have 911 null values in size

#### Analysis with datetimes

In [13]:
time_span = beauty_df["reviewTime"].max() - beauty_df["reviewTime"].min()

print ("Time span of the data: {}".format(time_span))

Time span of the data: 5084 days 00:00:00


In [14]:
beauty_df["year"] = beauty_df["reviewTime"].dt.year
beauty_df["month"] = beauty_df["reviewTime"].dt.month_name()
beauty_df["dayOfTheWeek"] = beauty_df["reviewTime"].dt.day_name()

In [17]:
year_count = beauty_df.groupby("year")["reviewText"].count().sort_values(ascending=False)

print("Amount of reviews per year: \n{}".format(year_count))

Amount of reviews per year: 
year
2016    1232
2015    1027
2017     825
2014     784
2013     756
2018     443
2012     111
2009      25
2011      20
2008      20
2007      13
2010       4
2006       2
2005       1
2004       1
Name: reviewText, dtype: int64


In [18]:
month_count = beauty_df.groupby("month")["reviewText"].count().sort_values(ascending=False)

print("Amount of reviews per month: \n{}".format(month_count))

Amount of reviews per month: 
month
August       598
September    550
April        548
March        518
July         464
May          445
February     413
January      410
December     352
November     338
October      320
June         308
Name: reviewText, dtype: int64


In [21]:
year_month_count = beauty_df.groupby(["year", "month"])["reviewText"].count().sort_values(ascending=False)

print ("Monthly breakdown of amount of reviews for each year: \n{}".format(year_month_count))

Monthly breakdown of amount of reviews for each year: 
year  month    
2017  September    244
2018  April        184
2016  July         177
      March        175
2015  August       159
                  ... 
2008  November       1
2009  February       1
      May            1
2018  July           1
2004  September      1
Name: reviewText, Length: 101, dtype: int64


In [24]:
yearly_rating = beauty_df.groupby("year")["productRating"].agg(["count","mean"]).sort_values(
    by="count", ascending=False)

print("Average rating per year: \n{}".format(yearly_rating))

Average rating per year: 
      count      mean
year                 
2016   1232  4.933442
2015   1032  4.932171
2017    825  4.713939
2014    784  4.891582
2013    756  4.880952
2018    443  3.724605
2012    111  4.927928
2009     25  4.680000
2008     20  3.200000
2011     20  4.400000
2007     13  4.230769
2010      4  5.000000
2006      2  5.000000
2004      1  5.000000
2005      1  5.000000


In [25]:
yearly_month_rating = beauty_df.groupby(["year", "month"])["productRating"].agg(["count","mean"]).sort_values(
    by="count", ascending=False)

print("Monthly breakdown for average rating per year: \n{}".format(yearly_month_rating))

Monthly breakdown for average rating per year: 
                count      mean
year month                     
2017 September    244  4.368852
2018 April        184  4.157609
2016 July         177  4.830508
     March        175  5.000000
2015 August       159  4.974843
...               ...       ...
2009 March          1  1.000000
     February       1  5.000000
2008 November       1  2.000000
     January        1  3.000000
2004 September      1  5.000000

[101 rows x 2 columns]


I created new columns for month, year and day of the week. I also grouped based on amount of reviews as well as average rating